# Autograd : automatic differentiation

http://pytorch.org/tutorials/beginner/blitz/autograd_tutorial.html#sphx-glr-beginner-blitz-autograd-tutorial-py

Pytorch에서 모든 신경망의 중심은 autograd 패키지이다.
간략하게 처음 맛보기 하고 첫 신경망을 학습시켜보자.

autograd 패키지는 텐서 객체 상의 모든 연산을 위한 자동 미분을 제공한다. define-by-run 프레임워크로서, 당신이 작성한 코드가 동작할때 역전파가 정의된다는걸 의미한다. 그리고 매 iteration이 달라질수있다는걸 의미한다.

몇 가지 예시로 개념을 살펴보자.



### Variable

autograd.Variable은 패키지의 중심 클래스 이다. 텐서 객체를 감싸고 정의된 연산의 거의 전부를 지원한다. 
당신의 연산이 완료되면 \.backward()를 호출할 수 있고 모든 그라디언트를 자동으로 계산가능하다.

당신은 \.data attribute를 통해 텐서객체의 raw data에 접근가능하다. 반면, gradient에 관해선 이 변수는 \.grad attribute로 통합한다.

![](http://pytorch.org/tutorials/_images/Variable.png)

한가지 더, autograd 구현에서 매우 중요한 클래스는 Function이다.

Variable과 Function 클래스는 밀접한 관련이 있고 연산의 완전한 순서가 기록된 무방향 그래프를 만든다.

각 변수는 Variable클래스를 구현하는 Function클래스를 참조하는 \.grad_fn attribute를 갖는다. (사용자가 구현한 Variable클래스 객체는 제외한다. 즉, grad_fn이 None이다.)

만약 당신이 미분 도함수를 계산하고싶다면, 당신은 Variable객체의 \.backward()를 호출가능하다. 만약 Variable객체가 scalar값이라면(즉, 엘리먼트가 하나라면), 당신은 backward()를 명시할 필요는 없다. 하지만 만약 다수의 엘리먼트를 갖는다면 grad_output 인자를 명시할 필요가있다.

In [1]:
import torch
from torch.autograd import Variable

### Create a Variable

In [2]:
x = Variable(torch.ones(2,2), requires_grad=True)
print(x)

Variable containing:
 1  1
 1  1
[torch.FloatTensor of size 2x2]



In [3]:
y = x + 2
print(y)

Variable containing:
 3  3
 3  3
[torch.FloatTensor of size 2x2]



In [4]:
print(y.grad_fn)

좀 더 y에 연산을 해보자.

In [5]:
z = y * y * 3
out = z.mean()

print(z, out)

Variable containing:
 27  27
 27  27
[torch.FloatTensor of size 2x2]
 Variable containing:
 27
[torch.FloatTensor of size 1]



이제 out.backward()로 역전파를 해보자.
out.backward(torch.Tensor([1,0])) 과 동치이다.

In [6]:
out.backward()

그라디언트를 출력해보자.$\frac{\partial(out)}{\partial x}$

In [7]:
print(x.grad)

Variable containing:
 4.5000  4.5000
 4.5000  4.5000
[torch.FloatTensor of size 2x2]



당신은 4.5의 행렬을 얻어야한다. 

out변수를 'o'라고 해보자. 

$o = \frac{1}{4}\sum_i z_i$

$z_i = 3(x_i + 2)^2 = z_i \bigr\rvert_{x_i=1} = 3(1+2)^2 = 27$

$\frac{\partial z}{\partial x} = 6(x_i+2)$

$\frac{\partial o}{\partial x} = \frac{6}{4}(x_i+2) = \frac{3}{2}(x_i+2)$

$\frac{\partial o}{\partial x} \bigr\rvert_{x_i=1} = \frac{9}{2} = 4.5$

autograd로 이런저런것도 가능하다.

In [19]:
x = torch.randn(3)
x = Variable(x, requires_grad=True)

y = x * 2
cnt = 0
while y.data.norm() < 1000:
    cnt += 1
    y = y * 2
    
print(y)

Variable containing:
  667.7912
-1068.5084
 -320.8718
[torch.FloatTensor of size 3]



In [20]:
cnt

8

In [22]:
gradients = torch.FloatTensor([0.1, 1.0, 0.0001])
y.backward(gradients)

print(x.grad)

Variable containing:
  51.2000
 512.0000
   0.0512
[torch.FloatTensor of size 3]



In [23]:
x

Variable containing:
 1.3043
-2.0869
-0.6267
[torch.FloatTensor of size 3]

In [24]:
y

Variable containing:
  667.7912
-1068.5084
 -320.8718
[torch.FloatTensor of size 3]

In [25]:
x.grad

Variable containing:
  51.2000
 512.0000
   0.0512
[torch.FloatTensor of size 3]